# 교차검증

In [176]:
# 테스트 세트를 사용하지 않고 모델이 과대적합인지, 과소적합인지, 즉 과적합 판단방법 <-- 머신러닝의 개념은 X
# 평가하고 측정하기 위한 하나의 도구일 뿐임
# 머신러닝의 성능을 높이기 위한 것이 아닌, 단지 보고서용으로 적절
# 데이터가 적거나 또는 머신러닝을 수행했을 때 과적합 발생 시 먼저 교차검증으로 평균치를 보여준 후
# 어느 정도 성능을 보인다고 명시하고
# 파라미터 튜닝을 통해 좀 더 성능을 끌어올려 보는 것 (검증의 개념)

# 성능이 너무 안 좋을 때에는 데이터를 의심해 봐야할 것...데이터 수집이 잘못되었다는 의심을 해볼 필요가 있고 혹은 전처리의 필요성이 있음
# 컬럼이 많다면 결정트리의 'feature 중요도'를 이용하여 상위 몇 개의 feature만 가지고 다시 수행해 볼 것

# 보통 거의 1 이 넘어가거나 0.99와 같은 값은 과적합일 가능성이 있음
# 0.90 ~ 0.95 정도의 수치가 가장 적절함
# 데이터의 수치를 낮췄는데도 test와 train의 값에 큰 차이가 없다면 과적합은 아님
# train과 test 두 값의 차이가 매우 클 때 과적합일 가능성이 크다
# 그러나 차이가 별로 나지 않아도 매우 낮으면 과소적합일 가능성이 크다

# cross_validate --> test데이터를 사용하지 않고 train 데이터만 가지고 과대/과소를 판단하기 위한 도구라고 생각하면 됨

In [177]:
import pandas as pd

In [178]:
df = pd.read_csv("https://raw.githubusercontent.com/bigdataleeky/python/main/wine.csv")
df.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [179]:
# 데이터 분류
# - 학습용 데이터
x = df.iloc[:,:-1]
x.head()

,alcohol,sugar,pH
0,9.4,1.9,3.51
1,9.8,2.6,3.20
2,9.8,2.3,3.26
3,9.8,1.9,3.16
4,9.4,1.9,3.51


In [180]:
y = df.iloc[:,-1]
y = y.astype(int)
y.head

<bound method NDFrame.head of 0       0
1       0
2       0
3       0
4       0
       ..
6492    1
6493    1
6494    1
6495    1
6496    1
Name: class, Length: 6497, dtype: int32>

In [181]:
from sklearn.model_selection import train_test_split

In [182]:
# 최종테스트 데이터
x_train, x_target, y_train, y_target = train_test_split(x,y,test_size=0.2, random_state=42)
x_train.shape, y_train.shape

((5197, 3), (5197,))

In [183]:
# 훈련용 데이터를 --> 훈련용 + 검증용 8:2
x_train2, x_target2, y_train2, y_target2 = train_test_split(x_train ,y_train ,test_size=0.2, random_state=42)

In [184]:
x_train2.shape, y_train2.shape

((4157, 3), (4157,))

In [185]:
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier(random_state=42)
dc.fit(x_train2, y_train2)
dc.score(x_train2, y_train2), dc.score(x_target2, y_target2)

(0.9971133028626413, 0.864423076923077)

# 위 모델을 가지고 과적합 평가, 즉 교차검증 해 보기

In [186]:
# GridSearchCV --> 하이퍼 파라미터 튜닝...모델 셀렉션의 하나
from sklearn.model_selection import cross_validate # 머신러닝의 기능을 조금 더 효율적으로 수행하기 위한 도구
score = cross_validate(dc, x_train, y_train) # dc를 가지고 최종테스트 데이터(x_train, y_train)으로 검증하기
score
# 다섯개로 나누어 교차하며 검증

{'fit_time': array([0.0229373 , 0.0169549 , 0.01296306, 0.01395988, 0.01196575]),
 'score_time': array([0.00498652, 0.00299239, 0.00399709, 0.00199699, 0.00199723]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [187]:
# type(score['test_score'])
# # numpy.ndarray
score['test_score']
# array가 붙으면 numpy임을 알 수 있음

array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])

In [188]:
score['test_score'].mean() # 1

0.855300214703487

In [189]:
import numpy as np
np.mean(score['test_score']) # 2
                             # 어느 것을 써도 무방

0.855300214703487

# 교차검증
## *(장점)*
### - 모든 데이터셋을 훈련과 평가에 활용 가능
### - 정확도 향상시킬 수 있음
### - 데이터가 부족할 때의 과적합을 해결 가능
### - 데이터 편중을 막을 수 있음
### - 보다 일반화된 모델 생성 가능

## *(단점)*
### - 여러 번 시도하기 때문에 시간이 과다 소요됨

In [190]:
# kfold 방식과 거의 유사
# 분류기를 통한 교차검증
# 분류모델이면 StratifiedKFold 사용
# 예측...즉 군집일 경우, all in other cases --> KFold 사용
from sklearn.model_selection import StratifiedKFold # 데이터 분류기 (분류 방법 정의)

In [191]:
# cross_validate 한 데이터를 다시 또 여러 번 반복
score = cross_validate(dc, x_train, y_train, cv=StratifiedKFold()) # cv : 분류기(분할 시 사용하는 유틸리티 함수)
score['test_score'].mean()
# 위와 거의 동일하지만 보다 더 정확하고 세밀히 분류한 것

0.855300214703487

In [192]:
dc.feature_names_in_ # 변수명 출력

array(['alcohol', 'sugar', 'pH'], dtype=object)

In [193]:
dc.feature_importances_ # 변수 중요도
# 가장 중요한 것 --> sugar

array([0.23614177, 0.50084785, 0.26301038])

In [194]:
# dc.feature_importances_.sort() --> 실행 안됨 (넘파이 데이터이기 때문에 아래와 같이 코드 작성해 주어야 함)
np.sort(dc.feature_importances_)[::-1] # 내림차순 정렬

array([0.50084785, 0.26301038, 0.23614177])

### 결정트리 사용 이유 : 정확한 분류
### 부가적인 기능 : feature의 중요도를 알 수 있음
###  ***만약 feature가 너무 많다면 feature_importance_의 값을 내림차순으로 정렬한 후 상위 몇 개의 feature만 추출하여 원하는 머신러닝에 적용하면 될 것

In [195]:
# 교차 검증으로 과적합을 피할 수 있다면 성능을 높이기 위해 하이퍼 파라미터 튜닝을 통해 성능 확인하기
dc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

In [196]:
# min_impurity_decrease 활용해 보기
params = {
    'min_impurity_decrease' : [0.0001, 0.001, 0.01, 0.01, 1, 10, 100]
}

In [197]:
# 튜닝...
from sklearn.model_selection import GridSearchCV

In [198]:
gs = GridSearchCV(dc, param_grid=params)

In [199]:
gs.fit(x_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'min_impurity_decrease': [0.0001, 0.001, 0.01, 0.01, 1,
                                                   10, 100]})

In [200]:
gs.best_params_
# 0.0001일 때가 가장 적합

{'min_impurity_decrease': 0.0001}

In [201]:
best_model = gs.best_estimator_

In [202]:
best_model.score(x_target2, y_target2)

0.9673076923076923

In [203]:
best_model.score(x_train, y_train), best_model.score(x_target, y_target)

(0.9615162593804117, 0.8653846153846154)

In [204]:
params = {
    'min_impurity_decrease' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'max_depth' : [3,5],
#     'min_samples_leaf' : np.arange(1, 5),
#     'min_samples_split' : np.arange(1, 5) --> 너~~무 오래 걸림
}

In [205]:
gs = GridSearchCV(dc, param_grid=params)
gs.fit(x_train, y_train)
gs.best_params_

{'max_depth': 5, 'min_impurity_decrease': 0.001}

In [206]:
best_model = gs.best_estimator_
best_model.score(x_train, y_train), best_model.score(x_target, y_target)

(0.8664614200500289, 0.8615384615384616)

In [207]:
# 한쪽으로 편중된, 과적합이 되는 것을 방지하기 위해 교차검증을 하는 것임
# 처음에 학습하기 위한 데이터를 분류하기 위해
# 랜덤으로 데이터를 섞은 후 8:2로 분류하고 2는 따로 보관해 둠
# 학습용 데이터 8을 다시 8:2로 분류하는 것을 반복하여 값을 측정해 나감 (머신러닝은 지속적으로 학습됨)
# 학습에 완전히 참여한 상태가 되면 학습한 것을 검증하기 위해 학습되지 않은, 한 번도 참여하지 않은 검증용 데이터 2로 검증 시행
# 이 분할 과정을 일일이 해 주는 것이 cross_validate : 머신러닝의 모델을 객관적으로 평가하기 위해 모델을 학습시키는 도구

# dc : 해당 머신러닝 dc(estimate)를 평가하기 위한 것
# 8에 해당하는 데이터 : x_train, y_train을 쪼개고 쪼개어 학습
# 학습 후 학습에 참여하지 않은 나머지 20%의 검증용 데이터에 문제를 주고 검증함

# 분류로는 StratifiedKFold (혹은 KFold)를 사용할 것
# 예측은 KFold, 분류는 StratifiedKFold를 쓰는 것이 적절

# Ensemble (앙상블)

### < 앙상블의 학습 기법 >
### Random Forest Algorithm (임의의 숲) - tree
### Extra Tree Algorithm
### Gradient Boosting Algorithm


In [208]:
# 정형데이터와 비정형데이터를 다룰 때 주로 사용
# 정형데이터를 다룰 때 머신러닝 중 '랜덤 포레스트'의 성능이 가장 뛰어남
# 성능을 높이기 위해 여러 개의 모델을 사용
# 같은 종류라 하더라도 데이터를 추출하기 때문에 다양한 성능을 낼 수 있음

# 정형데이터 - 구조가 정해져 있음
# 비정형 데이터 : 정의하거나 표현하기 힘든 불규칙적인 데이터(사진, 이미지, 음성파일 등)

In [209]:
# 랜덤 포레스트 알고리즘 : 결정트리(DecisionTree)를 무작위로 생성하여 숲을 구성

# 부트스트랩 샘플 : 무작위로 데이터를 추출하는 방법
#   - 데이터 추출 후 다시 원위치로 돌려 놓음 (랜덤한 값들도 잘 섞이게끔 하기 위함)

In [210]:
# 트리 알고리즘은 정규화 안함...(StandardScaling)
df = pd.read_csv("https://raw.githubusercontent.com/bigdataleeky/python/main/wine.csv")
df['class'] = df['class'].astype(int)
df.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0
1,9.8,2.6,3.20,0
2,9.8,2.3,3.26,0
3,9.8,1.9,3.16,0
4,9.4,1.9,3.51,0


In [211]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
x.shape, y.shape

((6497, 3), (6497,))

# 앙상블 - 랜덤 포레스트

In [212]:
from sklearn.ensemble import RandomForestClassifier

In [213]:
x_train, x_target, y_train, y_target = train_test_split(x, y, random_state=42)
x_train.shape, y_train.shape

((4872, 3), (4872,))

In [214]:
rfc = RandomForestClassifier(random_state=42)
rfc.fit(x_train, y_train)
rfc.score(x_train, y_train), rfc.score(x_target, y_target) 
# (0.9973316912972086, 0.8806153846153846) --> 과적합 발생

# 결정트리 : 과적합 자체는 피할 수는 없지만 성능 자체는 나아짐

(0.9973316912972086, 0.8812307692307693)

In [215]:
# cross_validate 사용해 보기
# 마지막 검증을 하지 않고 전체 학습 데이터에 해당하는 전체 평균값 살펴보기
score = cross_validate(rfc, x_train, y_train, cv=StratifiedKFold())
score

{'fit_time': array([0.45278931, 0.49069047, 0.580477  , 0.52357006, 0.48769307]),
 'score_time': array([0.03789687, 0.05684805, 0.03889585, 0.04288769, 0.0459137 ]),
 'test_score': array([0.88      , 0.90051282, 0.90349076, 0.89014374, 0.88295688])}

In [216]:
score['test_score'].mean()
# 랜덤 포레스트를 했을 때의 성능이 대략 0.8908048228294636 정도 나온다는 것을 알 수 있음
# 이전에는 8:2로 나누어 8만 가지고 cross_validate 하여 반복적으로 계속 나누어 학습한 뒤
# 마지막에 2를 가지고 실제 성능을 평가했음

# 현재는 전체데이터 전부를 학습하여 다시 나누지 않고 그 자체로 전체 평균을 살펴본 것

0.8914208392565683

In [217]:
# 전체데이터를 사용하는 이유
# - feature의 중요도를 알기 위해 
# - 랜덤 포레스트를 사용하는데, 크로스 벨리데이터를 이용해서 전체 학습데이터를 사용했다는 것은
# - 검증할 데이터는 없음 --> 중요 feature만 보기 위함

In [218]:
rfc.fit(x_train, y_train)
rfc.feature_importances_

array([0.23155241, 0.49706658, 0.27138101])

In [219]:
# Tree 알고리즘 그 자체로 구분도 할 수 있지만...
# Feature 엔지니어링에 주로 활용

In [220]:
rfc2 = RandomForestClassifier(oob_score=True)
rfc2.fit(x_train, y_train)
rfc2.feature_importances_ # 비교하려면 random_state값도 맞춰주어야 함

array([0.23406607, 0.49490174, 0.27103219])

# 앙상블 - 엑스트라 트리

In [221]:
# 평가할 때 교차검증을 하는 것이 보다 유리함
# 랜덤 포레스트 보다 무작위성이 큼
# 노드를 순차적 X --> 무작위로 분할하기 때문에 (랜덤 포레스트 보다) 계산속도가 빠름
# 데이터를 한꺼번에 많이, 조금 더 빨리 가져오는 것이 장점

In [222]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier()
score = cross_validate(et, x_train, y_train)

In [223]:
score

{'fit_time': array([0.43786287, 0.41488886, 0.49670267, 0.44378066, 0.40591931]),
 'score_time': array([0.05484867, 0.0588088 , 0.05185819, 0.04886699, 0.04889441]),
 'test_score': array([0.88820513, 0.89230769, 0.89835729, 0.89117043, 0.88090349])}

In [224]:
score['test_score'].mean()
# 이전과 거의 유사

0.8901888064023588

# 앙상블 - 그라디언트 부스팅

In [225]:
# 하나의 완전한 트리를 만드는 것 보다는,
# 깊이가 얕은 트리를 만들고 그 다음 트리는 이전 트리의 결점을 보완하는 방식의 앙상블
# 기본적으로 깊이가 3인 결정트리를 100개 생성함 -> 과적합에 강하고 높은 일반화 성능을 기대 (항상 X)
# 경사하강법(0314파일 참조)을 트리 앙상블에 적용한 것...손실함수 사용, 분류 --> 로지스틱 함수
#                                                                      회귀 --> 평균제곱와 MSE

In [226]:
from sklearn.ensemble import GradientBoostingClassifier
gd = GradientBoostingClassifier(random_state=42, learning_rate=0.3)
# learning rate(기울기)를 조정하면 성능을 조절할 수 있음
# 하나씩 점차 올리면서 일일이 확인하기에는 무리 --> 튜닝을 해야 함
score = cross_validate(gd, x_train, y_train, return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()

(0.876641288895909, 0.9146652520989097)

In [227]:
gd.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.3,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 42,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [228]:
from sklearn.model_selection import GridSearchCV # 튜닝
param = {
    'learning_rate' : np.arange(0.1, 1.5, 0.1)
}
gs = GridSearchCV(gd, param_grid=param)
gs.fit(x_train, y_train)

GridSearchCV(estimator=GradientBoostingClassifier(learning_rate=0.3,
                                                  random_state=42),
             param_grid={'learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4])})

In [229]:
gs.best_params_

{'learning_rate': 0.4}

In [230]:
model = gs.best_estimator_
score = cross_validate(model, x_train, y_train, return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()

(0.8770504922866319, 0.9233888719704268)

### 히스토그램 기반 그라디언트 부스팅
### 정형데이터를 다루는 머신러닝 알고리즘 중 가장 인기 多
### 입력 데이터의 특징을 보고 판단하여 구간을 나누어 작업
### X의 특성을 256개의 구간으로 나눔...트리를 구성할 때 최적의 노드가 되도록 분할 --> 효율과 속도 up
### 256개 중에서 한 개를 정답으로 삼아 사용하고 나머지 255개의 데이터를 학습용으로 사용

In [231]:
from sklearn.experimental import enable_hist_gradient_boosting # <-- 앙상블 계열이나 앙상블 패키지 X 
from sklearn.ensemble import HistGradientBoostingClassifier

In [232]:
hghc = HistGradientBoostingClassifier(random_state=42)
score = cross_validate(hghc, x_train, y_train, return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()

(0.8805410414363187, 0.9380129799494501)

In [233]:
hghc.get_params()

{'categorical_features': None,
 'early_stopping': 'auto',
 'l2_regularization': 0.0,
 'learning_rate': 0.1,
 'loss': 'auto',
 'max_bins': 255,
 'max_depth': None,
 'max_iter': 100,
 'max_leaf_nodes': 31,
 'min_samples_leaf': 20,
 'monotonic_cst': None,
 'n_iter_no_change': 10,
 'random_state': 42,
 'scoring': 'loss',
 'tol': 1e-07,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [234]:
param = {
    'learning_rate' : np.arange(0.1, 1.5, 0.1)
}
gs = GridSearchCV(hghc, param_grid=param)
gs.fit(x_train, y_train)

GridSearchCV(estimator=HistGradientBoostingClassifier(random_state=42),
             param_grid={'learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4])})

In [235]:
model = gs.best_estimator_
score = cross_validate(model, x_train, y_train, return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()
# 실제로 검증용이 높아졌다면 좋은 결과를 받은 것임

(0.8871097772863686, 0.9809627539727774)

In [236]:
gs.best_params_

{'learning_rate': 0.30000000000000004}

In [237]:
from sklearn.inspection import permutation_importance # 피쳐의 중요도

In [238]:
result = permutation_importance(model, x_train, y_train, random_state=42)
result.importances_mean, model.feature_names_in_ # 컬럼명 확인할 때는 변수명 X, 모델명으로 확인

(array([0.1365353 , 0.27557471, 0.14076355]),
 array(['alcohol', 'sugar', 'pH'], dtype=object))

In [239]:
result = permutation_importance(model, x_target, y_target, random_state=42)
result.importances_mean
# train과 target 모두 sugar column의 중요도가 가장 높게 나옴

array([0.05735385, 0.20455385, 0.05870769])

In [240]:
# model.score(x_train, y_train), model.score(x_target, y_target) --> 해줄 필요 없음

# 앙상블 > 그라디언트 부스팅 > XGBboost
### XGBboost -> 그라디언트 부스팅 알고리즘을 분산환경에서 실행할 수 있도록
### Begging / Boosting

In [241]:
# Begging
# - 각 모델에서 나온 값을 계산한 후, 최종 결과를 내는 방식

# Boosting
# - 순차적으로 이전 모델의 결과를 다음 모델에 반영하여 누적적으로 산출하는 방식)

In [242]:
# ! pip install xgboost

In [243]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
score = cross_validate(model, x_train, y_train, return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()

# Warning !
# score = cross_validate(model, x_train.to_numpy, y_train.to_numpy, return_train_score=True)

(0.8871097772863686, 0.9809627539727774)

In [244]:
! pip install lightgbm

In [251]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=42)
score = cross_validate(lgbm, x_train.to_numpy(), y_train.to_numpy(), return_train_score=True)
score['test_score'].mean(), score['train_score'].mean()

(0.8846461327857632, 0.9413484712095832)

In [252]:
# 정리
# 1. 랜덤 포레스트 : 부스트랩 생플 --> 전체 특징 중 일부 랜덤 선택
# 2. 엑스트라 트리 : 부스트랩 생플 사용 X --> 노드 랜덤 분할, 속도 up, 그러나 트리가 더 많이 소요
# 3. 그라디언트 부스팅 : deepth : 3인 tree의 집합, 성능 up, 그러나 속도가 느림
# 4. 히스토그램 기반 그라디언트 부스팅 : 가장 뛰어난 앙상블 기법으로,
#                                       계급 구간을 나누는데 255개로 나누어 사용, 정답은 단 한 개 (총 256개)
#                                       노드를 분할하는 게 가장 빠름
# 1 < 2 < 3 < 4 < XGBboost